In [10]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import unicodedata
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [3]:
y = df.pop('Labels')
np.unique(y)

array([0, 1, 2, 3, 4, 5])

## Building a pipeline

I remember from the nlp assignment that it was confusing to use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)... You can specify stopwords to remove, and a tokenizer, but need to remove punctuation and accents prior.

To get a better understanding of how it all works, I created a vocabulary to feed the CountVectorizer after sending the corpus through the cleaning_pipeline.

1. Lowercase
2. Remove punctuation
3. Remove stopwords
4. Remove accents
5. Lemmatize

In [4]:
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

def cleaning_pipeline(corpus, cleaner=WordNetLemmatizer):
    
    corpus = [row.lower() for row in corpus]
    print('Lowercase:')
    print(f'{corpus[:1]}')
    
    corpus = [punctuation_removal(row) for row in corpus]
    print('\nPunctuation Removed:')
    print(f'{corpus[:1]}')
    
    stop = stopwords.words('english')
    corpus = [' '.join([word for word in row.split() if word not in (stop)]) for row in corpus]
    print('\nStopwords Removed:')
    print(f'{corpus[:1]}')
    
    corpus = [remove_accents(row) for row in corpus]
    print('\nAccents Removed:')
    print(f'{corpus[:1]}')
    
    word_list = [word_tokenize(row) for row in corpus]
    print('\nTokenized:')
    print(f'{word_list[:1]}')
    
    stem_lemm = cleaner()
    stem_lemm_output = [' '.join([stem_lemm.lemmatize(word) for word in words]) for words in word_list]
    print('\nLemmatized:')
    print(f'{stem_lemm_output[:1]}')
    
    return stem_lemm_output

In [5]:
corpus = df['Text'].tolist()

docs_lem = cleaning_pipeline(corpus)

Lowercase:
['says the annies list political group supports third-trimester abortions on demand.']

Punctuation Removed:
['says the annies list political group supports thirdtrimester abortions on demand']

Stopwords Removed:
['says annies list political group supports thirdtrimester abortions demand']

Accents Removed:
['says annies list political group supports thirdtrimester abortions demand']

Tokenized:
[['says', 'annies', 'list', 'political', 'group', 'supports', 'thirdtrimester', 'abortions', 'demand']]

Lemmatized:
['say annies list political group support thirdtrimester abortion demand']


In [6]:
docs_lem[:5]

['say annies list political group support thirdtrimester abortion demand',
 'decline coal start started natural gas took started begin president george w bush administration',
 'hillary clinton agrees john mccain voting give george bush benefit doubt iran',
 'health care reform legislation likely mandate free sex change surgery',
 'economic turnaround started end term']

In [7]:
vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(docs_lem)
words = vectorizer.get_feature_names()
vect_df = pd.DataFrame(word_counts.toarray(), columns=words)
vect_df.head()

,005,01,02,025,03,04,047,05,06,07,...,zimmerman,zinn,zip,zippo,zombie,zone,zoning,zoo,zuckerberg,zuckerbergs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
vect_df['y'] = y
vect_df['tag'] = df['Text_Tag']
vect_df.head()

,005,01,02,025,03,04,047,05,06,07,0809,09,10,100,1000,10000,100000,100000000,100000the,1000plus,100billion,100million,100plus,100round,100th,100watt,100yearold,1010,1017,102000,103,103000,1033,1034,104,104000,10420,1045,105,106,106258,107,1070,10728,108,109,10932json,10billion,10in,10million,10percent,10th,10year,10yearold,11,110,1100,11000,11000squarefoot,11023json,110billion,110million,110th,111,11100,111th,112,112th,1130,113000,114,115,115000,116838763,119000,11900json,11collecting,11plus,11th,11yearold,12,120,1200,12000,120000,120member,12189json,123,124,124000,125,12500,125000,12670json,1270,128000,12808,128300,12853json,12million,12th,12year,12yearold,13,130,1300,13000,130000,132,133000,133600,1336page,134000,135,135000,136,137,138,139,1399,13m,13million,13th,13year,14,140,1400,14000,140000,141,141000,141600,142,142400,143,143000,144,144541,145,146,14th,15,150,1500,15000,150000,1508json,151,1525,153,153014,154,1540,1547,155,156,15600,157,15anhourproposal,15billion,15million,15th,16,160,1600,16000,160000,16000hour,16000year,161,162nd,163,1638json,164,165,1650,16500,166000,168,168k,16th,17,170,1700,17000,170000,172,1729000000,175,175587,176,176000,1761,177,1773,178,1789,179,1790,179000,1791,1792,1798,17th,18,180,1800,18000,180000,1800s,180day,181,1835,184,185,1850,186740,18972,18billion,18th,18year,18yearold,19,190,1900,190000,1900s,1904,1908,1912,1913,1915,1916,1917,1919,192000,1920s,1928,1929,1930,1930s,1935,1938,194,1940,1940s,1944,1947,1949,195,1950,1950s,1952,1954,1956,1959,196,1960,1960s,1961,1962,1964,1965,1968,1969,1970s,1971,1972,1973,1975,1976,1977,1978,1979,1980,1980s,1981,1982,1983,1984,1985,1986,1987,1988,1989,199,1990,1990every,1990s,1991,1992,1993,1994,1995,1996,1997,1998,1999,19percentraise,19yearolds,1absentee,1cent,1in12,1in2,1million,1one,1percent,1st,1trillion,20,200,2000,20000,200000,2001,20012012,2002,2003,2003and,2003bushtax,2004,200406,2005,2006,2007,200709,20072008,2008,2009,200911,200913,20092011,2009s,200mile,200year,2010,201011,20102011,20102014,2011,201113,20112012,2012,201213,2013,201315,20132015,2014,201415,20142015,2014s,2015,201516,201517,2015i,2016,2017,2019,2020,2021,2022,2023,2025,2030,2033,2037,2038,2040,205,2050,205075,2054th,2060s,2070,207000,2083,2096json,20billion,20million,20some,20th,20year,21,2100,21000,210000,211000,21174396,214000,21500,218029,219000,21million,21st,21yearold,22,220,2200,22000,220000,220000new,22000the,22081,223,223million,224,224000,226,228,229,22million,22yearold,23,230,2300,23000,230000,2301,2325000,2326,235,235000,236,237,238,23billion,23million,23rd,23rds,24,240,2400,24000,240000,2400pound,2403,2436json,244,244000,2465750,24700,248,2487,24th,25,250,2500,25000,250000,25000plus,252,257000,25th,25year,26,260,260000,265,265000,269,2694,27,270,27000,270000,27095000,271,272,2724json,27300,2733page,275,275000,277000,278,278000,28,2800,28000,280024,2812,282,283,284,2877,28776,287g,29,290,29000,292,293000,294,295,297x,298,29billion,2million,2nd,2plus,2trillion,2yearold,30,300,3000,30000,300000,300000000,3000squarefoot,300131,300k,300step,300vote,307,3072json,30day,30second,30th,31,310,31000,3128,313850,314,315000,3168json,319000,31st,32,320,32000,320000,321092,322,325,326,327,327653,328,33,33000,3304json,331,331247,332,33200,333485,335,339,33x,33year,34,340,3400,34000,3455,34th,34yearolds,35,350,3500,35000,350000,352,355,357,357000,358,35million,35th,36,360,36000,360000,3627json,3629,365,3666json,367,37,37000,370000,375,376,378000,37million,38,380,38000,380million,381,385,385json,38th,39,395,3c,3million,3rd,3to1,40,400,4000,40000,400000,400k,401k,401ks,401kstyle,4045,405,40billion,40hour,40th,40year,41,41000,41200,41500,4154,417,4191,41st,42,420,4200,42000,420000,425,4250,428,428000,42inch,43,43000,431,431000,43330,435,439,439000,43rd,44,440,4400,440000,442,443,444,44443,446,448,44th,45,450,4500,45000,450000,45000year,45193289,454260,45th,46,460,46000,4608json,4611json,466,468500,46th,46year,47,475,47500,47th,47that,48,4800,482,48700,489,48m,48th,49,49000,492,499,49

In [14]:
vect_df[['tag', 'y']]

,tag,y
0,abortion,1
1,"energy,history,job-accomplishments",2
2,foreign-policy,3
3,health-care,1
4,"economy,jobs",2
...,...,...
10235,"animals,elections",3
10236,elections,3
10237,"retirement,social-security",2
10238,"florida,foreign-policy",1


In [10]:
tfidfvect = TfidfVectorizer()
tfidf_vectorized = tfidfvect.fit_transform(docs_lem)
words_tfidf = tfidfvect.get_feature_names()
tfidf_df = pd.DataFrame(tfidf_vectorized.toarray(), columns=words_tfidf)
tfidf_df.head()

,005,01,02,025,03,04,047,05,06,07,...,zimmerman,zinn,zip,zippo,zombie,zone,zoning,zoo,zuckerberg,zuckerbergs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
tfidf_df['y'] = y
tfidf_df.head()

,005,01,02,025,03,04,047,05,06,07,...,zinn,zip,zippo,zombie,zone,zoning,zoo,zuckerberg,zuckerbergs,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
